In [78]:
import _sqlite3
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import re
import datetime

import importlib
import common.lottery_results
importlib.reload(common.lottery_results)
from common.lottery_results import LotteryResultsCRUD

import inspect

pd.set_option('display.width', 4500)

In [79]:
sqlite_db_name = 'lottery_checker.sqlite'
connection = _sqlite3.connect(sqlite_db_name)
crud = LotteryResultsCRUD(sqlite_db_name)

method = crud.check_and_insert
num_args = len(inspect.signature(method).parameters)
print(f"The method requires {num_args} arguments.")

The method requires 1 arguments.


In [ ]:
query = "Select * from lottery_meta where active = 'yes'"

df = pd.read_sql_query(query, connection)

for index, meta in df.iterrows():
    
    # calculate the ending year, this applies to the Euro millions as changed format
    # build the year range
    if meta.lottery_end_year is None or (isinstance(meta.lottery_end_year, float) and math.isnan(meta.lottery_end_year)):
        end_year = datetime.datetime.now().year + 1
    else:
        end_year = math.floor(meta.lottery_end_year + 1)
    year_range = list(range(meta.lottery_start_year, end_year))

    lottery_slug = meta.lottery_slug
    lottery_id = meta.lottery_id
    col_string = meta.lottery_archive_columns
    cols = col_string.split('|')
    cols = ['lottery_id'] + cols
    cols = ['draw_id'] + cols

    print(cols)
    
    # loop over year_range to scrape year results
    for year in year_range:
               
        base_url = 'https://www.lottery.co.uk/{}/results/archive-{}'.format(lottery_slug, year)
        
        #print('-- YEAR: {}'.format(year))
        #print(base_url)
        
        response = requests.get(base_url)
        html = response.content

        parts = lottery_slug.split('-')
        html_table_class = parts[0] + ''.join(word.capitalize() for word in parts[1:])

        soup = BeautifulSoup(html, "html.parser")
        table = soup.find("table", class_=html_table_class)
        
        if table:
            
            rows = table.find_all("tr")
            
            for idx, row in enumerate(rows[1:], start=1):
                      
                cells = row.find_all(["td", "th"])
                row_data = [cell.text.strip() for cell in cells]
                
                # Extract the href value from the first cell (assuming it's a link)
                first_cell = cells[0]
                href_value = first_cell.find('a')['href']
                
                # Split the href value to extract the date part
                parts = href_value.split('-')
                date_str = '-'.join(parts[-1:] + parts[-3:-1])
                date_key = lottery_id + '|' + date_str
            
                # Add the extracted date as a new item in the dictionary
                data_dict = {key: value for key, value in zip(cols, row_data)}
                data_dict['lottery_id'] = lottery_id
                data_dict['draw_id'] = date_key
                data_dict['draw_date_str'] = date_str
                
                data_df = pd.DataFrame(columns=cols)
                
                # Append the dictionary to the DataFrame
                data_df = data_df.append(data_dict, ignore_index=True)
                #print(data_df.columns)
                
                # Melt the DataFrame while keeping 'draw_id' and 'lottery_id' columns
                melted_df = pd.melt(data_df, id_vars=['draw_id', 'lottery_id', 'draw_date_str'], var_name='draw_column', value_name='draw_values')
                        
                # check the db.lottery_results to see if [draw_id] exists
                # draw_id, draw_date, draw_column, draw_values, lottery_id
                crud.check_and_insert(melted_df)
                
                del melted_df
                del data_df
    
    print('---------------------')


In [119]:
lottery_slug = 'set-for-life'
year = '2019'
lottery_archive_columns = 'draw_date|results|winners'

base_url = 'https://www.lottery.co.uk/{}/results/archive-{}'.format(lottery_slug, year)

print(base_url)

response = requests.get(base_url)
html = response.content

parts = lottery_slug.split('-')
html_table_class = parts[0] + ''.join(word.capitalize() for word in parts[1:])

print(html_table_class)
html_table_class = 'table {} mobFormat'.format(html_table_class)
col_string = lottery_archive_columns
cols = col_string.split('|')
cols = ['lottery_id'] + cols
cols = ['draw_id'] + cols
print(html_table_class)

soup = BeautifulSoup(html, "html.parser")
table = soup.find("table", class_=html_table_class)

print(table)

if table:
    
    rows = table.find_all("tr")
    
    for idx, row in enumerate(rows[1:], start=1):
              
        cells = row.find_all(["td", "th"])
        row_data = [cell.text.strip() for cell in cells]
        
        # Extract the href value from the first cell (assuming it's a link)
        first_cell = cells[0]
        href_value = first_cell.find('a')['href']
        
        # Split the href value to extract the date part
        parts = href_value.split('-')
        date_str = '-'.join(parts[-1:] + parts[-3:-1])
        date_key = 'lo|' + date_str
    
        # Add the extracted date as a new item in the dictionary
        data_dict = {key: value for key, value in zip(cols, row_data)}
        data_dict['lottery_id'] = 'lo'
        data_dict['draw_id'] = date_key
        data_dict['draw_date_str'] = date_str
               
        data_df = pd.DataFrame(columns=cols)
        
        # Append the dictionary to the DataFrame
        data_df = data_df.append(data_dict, ignore_index=True)
        print(data_df.columns)
        
        # Melt the DataFrame while keeping 'draw_id' and 'lottery_id' columns
        melted_df = pd.melt(data_df, id_vars=['draw_id', 'lottery_id', 'draw_date_str'], var_name='draw_column', value_name='draw_values')
        
        print(melted_df)

        # check the db.lottery_results to see if [draw_id] exists
        # draw_id, draw_date, draw_column, draw_values, lottery_id
        crud.check_and_insert(melted_df)
        
        del melted_df
        del data_df
        
        # add to database
        

https://www.lottery.co.uk/set-for-life/results/archive-2019
setForLife
table setForLife mobFormat
<table class="table setForLife mobFormat" style="width: 100%;">
<thead>
<tr>
<th style="white-space: nowrap;">Draw Date</th>
<th>Results</th>
<th>Winners</th>
</tr>
</thead>
<tr>
<td class="noBefore" style="text-align: center !important;"><a class="smallerHeading nowrap" href="/set-for-life/results-30-12-2019" title="View the Prize Breakdown and Draw Details for Monday 30th December, 2019">Monday 30th December 2019</a></td>
<td class="noBefore" style="text-align: center !important; white-space: nowrap;">
<div class="result small setForLife-ball">8</div>
<div class="result small setForLife-ball">18</div>
<div class="result small setForLife-ball">21</div>
<div class="result small setForLife-ball">23</div>
<div class="result small setForLife-ball">44</div>
<div class="result small setForLife-life-ball">4</div>
</td>
<td class="noBefore" style="text-align: center !important; white-space: nowra

In [110]:
query = "Select * from lottery_meta where active = 'yes'"

df = pd.read_sql_query(query, connection)

for index, meta in df.iterrows():
    
    # define variables


https://www.lottery.co.uk/set-for-life/results/archive-2019
<Response [200]>
setForLife
